# MySQL-Datenbank "la mediterranea" 

## Import des Python Moduls, Herstellung der Verbindung

In [ ]:
import mysql.connector
import xml.etree.ElementTree as ET

# Funktion zur Bereinigung von HTML-Entities in der XML-Datei
def clean_html_entities(xml_content):
    return xml_content.replace('&#39;', "'").replace('&#46;', '.').replace('&#45;', '-')

# XML-Datei lesen und bereinigen
xml_file = 'v078803.xml'
with open(xml_file, 'r', encoding='utf-8') as file:
    xml_content = file.read()

cleaned_xml_content = clean_html_entities(xml_content)

# Bereinigung von ungültigen Tags in der XML
cleaned_xml_content = cleaned_xml_content.replace('<No Doubt/Artist>', '<Artist>No Doubt</Artist>')

# Parsen der bereinigten XML-Datei
try:
    root = ET.fromstring(cleaned_xml_content)
except ET.ParseError as e:
    print("Fehler beim Parsen der XML-Datei:", e)
    exit(1)

# MySQL-Datenbankverbindung initialisieren
db_config = {
    'user': 'v077803',
    'password': 'HibaIrinaPuyaHeiko2024_',
    'host': 'localhost',
    'database': 'v077803',
}

db = mysql.connector.connect(**db_config)
cursor = db.cursor()

# Tabelle erstellen
cursor.execute('''CREATE TABLE IF NOT EXISTS tracks (
    track_id INT PRIMARY KEY,
    track_name VARCHAR(255),
    track_artist VARCHAR(255),
    track_composer VARCHAR(255),
    track_album VARCHAR(255),
    track_grouping VARCHAR(255),
    track_genre VARCHAR(255),
    track_kind VARCHAR(255),
    track_size INT,
    track_total_time INT,
    track_disc_number INT,
    track_number_disc INT,
    track_year INT,
    track_average_bpm FLOAT,
    track_date_added DATE,
    track_bit_rate INT,
    track_sample_rate INT,
    track_comments TEXT,
    track_play_count INT,
    track_rating INT,
    track_remixer VARCHAR(255),
    track_tonality VARCHAR(255),
    track_label VARCHAR(255),
    track_mix VARCHAR(255)
)''')

# Daten einfügen
insert_query = '''INSERT INTO tracks (
    track_id, track_name, track_artist, track_composer, track_album, track_grouping, track_genre, track_kind, track_size, track_total_time, track_disc_number, track_number_disc, track_year,
    track_average_bpm, track_date_added, track_bit_rate, track_sample_rate, track_comments, track_play_count, track_rating, track_remixer, track_tonality, track_label, track_mix
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

for record in root.findall('record'):
    track_data = (
        int(record.find('TrackID').text),
        record.find('Name').text,
        record.find('Artist').text,
        record.find('Composer').text,
        record.find('Album').text,
        record.find('Grouping').text,
        record.find('Genre').text,
        record.find('Kind').text,
        int(record.find('Size').text) if record.find('Size').text else None,
        int(record.find('TotalTime').text) if record.find('TotalTime').text else None,
        int(record.find('DiscNumber').text) if record.find('DiscNumber').text else None,
        int(record.find('TrackNumber').text) if record.find('TrackNumber').text else None,
        int(record.find('Year').text) if record.find('Year').text else None,
        float(record.find('AverageBpm').text.replace('&#46;', '.')) if record.find('AverageBpm').text else None,
        record.find('DateAdded').text,
        int(record.find('BitRate').text) if record.find('BitRate').text else None,
        int(record.find('SampleRate').text) if record.find('SampleRate').text else None,
        record.find('Comments').text,
        int(record.find('PlayCount').text) if record.find('PlayCount').text else None,
        int(record.find('Rating').text) if record.find('Rating').text else None,
        record.find('Remixer').text,
        record.find('Tonality').text,
        record.find('Label').text,
        record.find('Mix').text
    )
    cursor.execute(insert_query, track_data)

db.commit()
cursor.close()
db.close()

print("Track-Daten erfolgreich in die Datenbank geschrieben.")

## MySQL-Datenbank

<https://v078803.kasserver.com/mysqladmin/>  
Benutzername: v078803  
Passwort: HibaIrinaPuyaHeiko2024_  
Datenbankname: v078803


In [ ]:
import mysql.connector
from mysql.connector import Error

### lokale Datenbank
config = {
    'user': 'v078803',
    'password': 'HibaIrinaPuyaHeiko2024_',
    'host': 'localhost',
    'unix_socket': '/Applications/MAMP/tmp/mysql/mysql.sock',
    'database': 'v078803',
    'raise_on_warnings': True
}


### Server Datenbank
""" config = {
    'user': 'v078803',
    'password': 'HibaIrinaPuyaHeiko2024_',
    'host': 'v078803.kasserver.com',
    'database': 'v078803',
    'raise_on_warnings': True
} """

# connection zum MySQL-Server herstellen
db_connection = mysql.connector.connect(**config)

print("Verbunden mit:", db_connection.get_server_info())


### Erstellen der Tabelle playlist

||__playlist__|
|---|---|
|PK|track_id|
||track_artist|
||track_bpm|
||track_device|
||track_label|
||track_timestamp|
|__FK__|rekordbox_id|

In [ ]:
try:
    # connection zum MySQL-Server herstellen
    db_connection = mysql.connector.connect(**config)
    if db_connection.is_connected():
        print("Verbunden mit:", db_connection.get_server_info())
        print("Erstelle die Tabellen...")
        db_cursor = db_connection.cursor()
        ### Tabelle 'conditions' erstellen
        sql_query = """
            CREATE TABLE playlist (
                track_id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
                track_artist VARCHAR(255),
                track_bpm FLOAT,
                track_device VARCHAR(255),
                track_genre VARCHAR(255),
                track_key VARCHAR(255),
                track_label VARCHAR(255),
                track_title VARCHAR(255),
                track_timestamp TIMESTAMP,
                rekordbox_id INT NOT NULL
            )
        """
        db_cursor.execute(sql_query)
        print("Tabelle 'playlist' erfolgreich erstellt.")
    else:
        print("Verbindung konnte nicht hergestellt werden.")
except Error as e:
    print("Fehler beim Verbinden mit MySQL", e)
finally:
    if (db_connection.is_connected()):
        db_cursor.close()
        db_connection.close()
        print("MySQL-Verbindung ist geschlossen.")

### Playlisten vom Tracked Update Expression Trigger von BLT

In [ ]:
import socket
import threading
import json
import mysql.connector
from mysql.connector import Error

# Globale Variablen für Track-Daten und Track-Historie
track_data = {}
track_history = {}

# Placeholder für Socket.IO (hier muss die tatsächliche Socket.IO-Initialisierung erfolgen)
class Socket:
    def emit(self, event, data):
        print(f"Event: {event}, Data: {data}")

socketio = Socket()

def convert_time(milliseconds):
    """Konvertiert Millisekunden in ein Minuten:Sekunden-Format."""
    seconds = milliseconds / 1000
    minutes = int(seconds // 60)
    seconds = int(seconds % 60)
    return f"{minutes:02}:{seconds:02}"

def update_track_info(json_data):
    """Aktualisiert die Track-Informationen, sendet sie über Socket.IO und speichert sie in der MySQL-Datenbank und in der Historie."""
    global track_data, track_history
    track_data = {
        "track_artist": json_data.get("artist"),
        "track_bpm": round(json_data.get("bpm"), 2) if json_data.get("bpm") is not None else None,
        "track_device": json_data.get("device"),
        "track_genre": json_data.get("track_genre"),
        "track_key": json_data.get("key"),
        "track_label": json_data.get("label"),
        "track_timestamp": json_data.get("timestamp"),
        "track_title": json_data.get("title"),
        "rekordbox_id": json_data.get("id")
    }
    socketio.emit('update', track_data)
    save_to_database(track_data)
    add_to_history(track_data)

def save_to_database(track_data):
    """Speichert die Track-Daten in der MySQL-Datenbank."""
    try:
        db_connection = mysql.connector.connect(**config)
        
        if db_connection.is_connected():
            cursor = db_connection.cursor()
            insert_query = """
                INSERT INTO playlist (track_artist, track_bpm, track_device, track_genre, track_key, track_label, track_timestamp, track_title, rekordbox_id)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            track_data_tuple = (
                track_data["track_artist"],
                track_data["track_bpm"],
                track_data["track_device"],
                track_data["track_genre"],
                track_data["track_key"],
                track_data["track_label"],
                track_data["track_timestamp"],
                track_data["track_title"],
                track_data["rekordbox_id"]
            )
            cursor.execute(insert_query, track_data_tuple)
            db_connection.commit()
            print("Track-Daten erfolgreich in die Datenbank geschrieben.")
    except Error as e:
        print("Fehler beim Verbinden mit der MySQL-Datenbank", e)
    finally:
        if (db_connection.is_connected()):
            cursor.close()
            db_connection.close()
            print("MySQL-Verbindung geschlossen.")

def add_to_history(track_data):
    """Fügt die Track-Daten der Track-Historie hinzu."""
    global track_history
    track_id = track_data["rekordbox_id"]
    track_history[track_id] = track_data
    print("Track-Daten erfolgreich zur Historie hinzugefügt:", track_history)

def socket_receiver():
    """Empfängt UDP-Pakete und verarbeitet die JSON-Daten."""
    # Erstellen eines UDP-Sockets
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    port = 7001
    sock.bind(('', port))
    print("Socket verbunden mit Port", port)
    receiving = True

    try:
        while receiving:
            print('\nWarten auf eine Nachricht')
            data, address = sock.recvfrom(4096)
            print('Erhaltene {} Bytes von {}'.format(len(data), address))

            try:
                json_data = json.loads(data.decode('utf-8'))  # Daten dekodieren
                print("JSON-Daten erfolgreich dekodiert:", json_data)
                update_track_info(json_data)
            except json.JSONDecodeError as e:
                print("Fehler beim Dekodieren der JSON-Daten:", e)
    finally:
        sock.close()  # Socket schließen

if __name__ == '__main__':
    receiver_thread = threading.Thread(target=socket_receiver)
    receiver_thread.start()